In [1]:
import os
import numpy as np
from matplotlib import image
from shapely.wkt import loads
import json
import cv2 as cv

In [2]:
#these are in the data.zip
img_dir = "./Images"
label_dir = './Json'
crop_size = 128
n_samples = 100
out_dir = './dataset'

In [3]:
colours_dict = {'no-damage':[0,1,0],
           'minor-damage':[0,1,0],
           #this is a little contentious and may not have helped since we're labelling major damage as destroyed to rebalance the classes
           'major-damage':[1,0,0],
           'destroyed':[1,0,0]
           }

for _filename in os.listdir(img_dir):
    _filename = os.path.splitext(_filename)[0]
    if "post" in _filename:
        _label_file = open(f'{label_dir}/{_filename}.json')
        _label_data =json.load(_label_file)
        _img = image.imread(f'{img_dir}/{_filename}.png')
        _polygons = []
        out = np.zeros_like(_img)
        out[:,:,-1]=1
        #Plotting all of the objects into the output arrays using openCV 
        for _obj in _label_data['features']['xy']:
            _polygon = loads(_obj['wkt'])
            _polygons.append(_polygon)
            _polygon = np.array(list(_polygon.exterior.coords),dtype=np.int32)
            try:
                #colour based on damage level
                colour = colours_dict[_obj['properties']['subtype']]
            except KeyError:
                #sometimes the damage label is incorrect
                colour = [0,1,0]
            out = cv.fillPoly(out,[_polygon],colour)
        
        #generate a number of croppings from each satellite image
        for n in range(n_samples):
            x = np.random.randint(0,_img.shape[0]-crop_size) + np.array([0,crop_size])
            y = np.random.randint(0,_img.shape[1]-crop_size) + np.array([0,crop_size])
            sample = _img[x[0]:x[1],y[0]:y[1]] 
            mask = out[x[0]:x[1],y[0]:y[1]]
            
            #rebalancing the data a little bit otherwise a vast majority was empty
            if np.sum(mask[:,:,0]) > 0:
                image.imsave(f"{out_dir}/mask/{_filename}_{n}.jpeg", mask)
                image.imsave(f"{out_dir}/sample/{_filename}_{n}.jpeg", sample)
            else:
                if np.random.randint(0,5) < 1: 
                    image.imsave(f"{out_dir}/mask/{_filename}_{n}.jpeg", mask)
                    image.imsave(f"{out_dir}/sample/{_filename}_{n}.jpeg", sample)